In [1]:
import os, sys
import numpy as np
import pandas as pd
import flopy

In [4]:
def forward_model():
    try:
        os.remove("heads_out.csv")
    except:
        pass
    
    indx = pd.read_csv('index.csv')['idx'].values.astype(int)   
    
    
    x = np.loadtxt('inputx.dat')
    x = np.power(10.0, x)
    mf = flopy.modflow.Modflow.load(r'flow_1d.nam', model_ws = 'Beta_03' )
    
    hk = mf.upw.hk.array.copy()
    hk = x[np.newaxis, np.newaxis, :]
    mf.upw.hk = hk
    mf.upw.write_file()

    basefolder = os.getcwd()
    os.chdir("Beta_03")
    os.system("mfnwt.exe flow_1d.nam")
    os.chdir(basefolder)

    hds = flopy.utils.HeadFile(os.path.join('Beta_03', 'flow_1d.hds'))
    wl = hds.get_data(totim=1.0)
    wl = wl.squeeze()
    y =wl[indx]                                                 # model maping  
    
    out = pd.DataFrame()
    out['y'] = y
    out.to_csv('heads_out.csv', index_label = 'id')             # write model output
    
if __name__ == "__main__":
    forward_model()

In [1]:
!jupyter nbconvert --to script c_Beta_Script_02_at_FOLDER_02.ipynb

[NbConvertApp] Converting notebook c_Beta_Script_02_at_FOLDER_02.ipynb to script
[NbConvertApp] Writing 1077 bytes to c_Beta_Script_02_at_FOLDER_02.py
